In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

import re
import nltk
from nltk.corpus import stopwords
from collections import Counter

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError


In [2]:
def clean_text(text):
    # Remove special characters and numbers
    text = re.sub('[^a-zA-Z]', ' ', text)

    # Convert text to lowercase and tokenize words
    words = text.lower().split()

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Join words back into a single string
    cleaned_text = ' '.join(words)
    
    return cleaned_text



In [4]:
# Load datasets
fake_politifact = pd.read_csv('politifact_fake.csv')
fake_gossipcop = pd.read_csv('gossipcop_fake.csv')
abcnews = pd.read_csv('abcnews-date-text.csv')

# Assign labels: 0 for fake news, 1 for real news
fake_politifact['label'] = 0
fake_gossipcop['label'] = 0
abcnews['label'] = 1

# Preprocess the dataset
abcnews = abcnews.sample(5000)  # Take the first 5,000 headlines
abcnews.rename(columns={'headline_text': 'title'}, inplace=True)  # Rename the column to match other datasets
abcnews['cleaned_title'] = abcnews['title'].apply(clean_text)  # Clean the headlines

# Concatenate datasets
data = pd.concat([fake_politifact, abcnews, fake_gossipcop], axis=0, ignore_index=True)
data['cleaned_title'] = data['title'].apply(clean_text)

In [5]:
# Split data into training and testing sets (80% train, 20% test)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [6]:
train_data.shape

(8604, 7)

In [7]:
train_data.head()


,id,news_url,title,tweet_ids,label,publish_date,cleaned_title
9537,gossipcop-2717691939,www.townandcountrymag.com/society/a15845582/wh...,Latest News on Fired NBC Today Show Anchor,974979120537985024\t1007020604099301377\t10307...,0,NaN,latest news fired nbc today show anchor
10425,gossipcop-7842742178,www.ok.co.uk/celebrity-news/1234596/margot-rob...,'Pregnant' Margot Robbie 'expecting child' wit...,816708608469663744\t816708622432337924\t816708...,0,NaN,pregnant margot robbie expecting child tom ack...
4339,NaN,NaN,firefighters hope weather aids efforts to tackle,NaN,1,20051011.0,firefighters hope weather aids efforts tackle
353,politifact14116,www.usatoday.com/story/gameon/2012/11/12/lance...,Lance Armstrong's defiant Twitter photo shows ...,NaN,0,NaN,lance armstrong defiant twitter photo shows re...
456,NaN,NaN,melbourne man hospitalised after stabbing,NaN,1,20070325.0,melbourne man hospitalised stabbing


In [8]:
test_data.shape

(2151, 7)

In [9]:
test_data.head()

,id,news_url,title,tweet_ids,label,publish_date,cleaned_title
9043,gossipcop-8776120919,hollywoodlife.com/2017/07/28/george-clooney-te...,George Clooney Terrified Over Safety Of His Tw...,891131523562426369\t891132638261649408\t891133...,0,NaN,george clooney terrified safety twins photogra...
1501,NaN,NaN,study to investigate flood protection measures,NaN,1,20051107.0,study investigate flood protection measures
6044,gossipcop-5750046920,www.accessonline.com/articles/ariel-winters-bo...,Ariel Winter's Boyfriend Levi Meaden Cheekily ...,939265361869930496\t939266680051716096\t939366...,0,NaN,ariel winter boyfriend levi meaden cheekily gr...
10720,gossipcop-6421829445,www.telegraph.co.uk/news/2017/07/27/justin-bie...,Justin Bieber accidentally hits photographer w...,890469931137081344\t890477494763114496\t890480...,0,NaN,justin bieber accidentally hits photographer p...
10196,gossipcop-9533859944,metro.co.uk/2018/04/16/nicole-kidman-wants-kei...,Nicole Kidman wants Keith Urban to join cast o...,967193199151386625,0,NaN,nicole kidman wants keith urban join cast big ...


In [10]:
def load_glove_vectors(glove_file):
    word_vectors = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            word_vectors[word] = vector
    return word_vectors

glove_vectors = load_glove_vectors('glove.6B.50d.txt')


In [11]:
def weighted_average_vectors(docs, vectorizer, word_vectors, dim=50):
    # Calculate the tf-idf weights for the given documents
    tfidf_matrix = vectorizer.fit_transform(docs)

    # Initialize an empty matrix to store the weighted average vectors
    weighted_vectors = np.zeros((len(docs), dim))

    # Iterate through the documents and compute the weighted average vector for each
    for i, doc in enumerate(docs):
        words = doc.split()
        weighted_sum = np.zeros(dim)
        total_weight = 0

        for word in words:
            if word in word_vectors and word in vectorizer.vocabulary_:
                vector = word_vectors[word]
                weight = tfidf_matrix[i, vectorizer.vocabulary_[word]]
                weighted_sum += weight * vector
                total_weight += weight

        if total_weight != 0:
            weighted_vectors[i] = weighted_sum / total_weight

    return weighted_vectors


In [12]:
vectorizer = TfidfVectorizer(max_features=10000)
X_train = weighted_average_vectors(train_data['cleaned_title'], vectorizer, glove_vectors)
X_test = weighted_average_vectors(test_data['cleaned_title'], vectorizer, glove_vectors)

y_train = train_data['label'].values
y_test = test_data['label'].values


In [13]:
def create_autoencoder(input_dim, encoding_dim, hidden_layers):
    # Encoder
    encoder_input = Input(shape=(input_dim,))
    encoder_layers = [Dense(hidden_layers[0], activation='relu')(encoder_input)]

    for layer_size in hidden_layers[1:]:
        encoder_layers.append(Dense(layer_size, activation='relu')(encoder_layers[-1]))

    # Encoded representation
    encoded = Dense(encoding_dim, activation='relu')(encoder_layers[-1])

    # Decoder
    decoder_layers = [Dense(hidden_layers[-1], activation='relu')(encoded)]

    for layer_size in reversed(hidden_layers[:-1]):
        decoder_layers.append(Dense(layer_size, activation='relu')(decoder_layers[-1]))

    # Reconstruction
    decoder_output = Dense(input_dim, activation='linear')(decoder_layers[-1])

    # Build the autoencoder model
    autoencoder = Model(encoder_input, decoder_output)
    return autoencoder


In [14]:
input_dim = X_train.shape[1]
encoding_dim = 32
hidden_layers = [128, 64]

autoencoder = create_autoencoder(input_dim, encoding_dim, hidden_layers)
autoencoder.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 50)]              0         
                                                                 
 dense (Dense)               (None, 128)               6528      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 64)                2112      
                                                                 
 dense_4 (Dense)             (None, 128)               8320      
                                                                 
 dense_5 (Dense)             (None, 50)                6450  

In [15]:
# Compile the model
optimizer = Adam(learning_rate=0.001)
loss = MeanSquaredError()
autoencoder.compile(optimizer=optimizer, loss=loss)

# Train the model
real_headlines = X_train[y_train == 1]  # filter real news headlines
epochs = 100
batch_size = 32
autoencoder.fit(real_headlines, real_headlines, epochs=epochs, batch_size=batch_size, validation_split=0.1)


Epoch 1/100
113/113 [==============================] - 2s 10ms/step - loss: 0.1062 - val_loss: 0.0698
Epoch 2/100
113/113 [==============================] - 0s 4ms/step - loss: 0.0591 - val_loss: 0.0499
Epoch 3/100
113/113 [==============================] - 0s 4ms/step - loss: 0.0456 - val_loss: 0.0420
Epoch 4/100
113/113 [==============================] - 0s 4ms/step - loss: 0.0395 - val_loss: 0.0383
Epoch 5/100
113/113 [==============================] - 0s 4ms/step - loss: 0.0354 - val_loss: 0.0344
Epoch 6/100
113/113 [==============================] - 0s 4ms/step - loss: 0.0324 - val_loss: 0.0323
Epoch 7/100
113/113 [==============================] - 0s 4ms/step - loss: 0.0306 - val_loss: 0.0314
Epoch 8/100
113/113 [==============================] - 0s 4ms/step - loss: 0.0296 - val_loss: 0.0303
Epoch 9/100
113/113 [==============================] - 0s 4ms/step - loss: 0.0289 - val_loss: 0.0298
Epoch 10/100
113/113 [==============================] - 0s 4ms/step - loss: 0.0283 - val_l

113/113 [==============================] - 0s 4ms/step - loss: 0.0190 - val_loss: 0.0229
Epoch 82/100
113/113 [==============================] - 0s 4ms/step - loss: 0.0189 - val_loss: 0.0232
Epoch 83/100
113/113 [==============================] - 0s 4ms/step - loss: 0.0190 - val_loss: 0.0232
Epoch 84/100
113/113 [==============================] - 0s 4ms/step - loss: 0.0189 - val_loss: 0.0230
Epoch 85/100
113/113 [==============================] - 0s 4ms/step - loss: 0.0190 - val_loss: 0.0229
Epoch 86/100
113/113 [==============================] - 0s 4ms/step - loss: 0.0189 - val_loss: 0.0228
Epoch 87/100
113/113 [==============================] - 0s 4ms/step - loss: 0.0189 - val_loss: 0.0231
Epoch 88/100
113/113 [==============================] - 0s 4ms/step - loss: 0.0189 - val_loss: 0.0226
Epoch 89/100
113/113 [==============================] - 0s 4ms/step - loss: 0.0188 - val_loss: 0.0232
Epoch 90/100
113/113 [==============================] - 0s 4ms/step - loss: 0.0189 - val_loss: 

In [16]:
# Reconstruct headlines in the test set
X_test_reconstructed = autoencoder.predict(X_test)

# Calculate the reconstruction errors
reconstruction_errors = np.mean((X_test - X_test_reconstructed)**2, axis=1)


68/68 [==============================] - 0s 2ms/step


In [17]:
from sklearn.metrics import confusion_matrix

def find_optimal_threshold(errors, y_true):
    min_diff = float('inf')
    optimal_threshold = 0

    for threshold in np.linspace(errors.min(), errors.max(), num=1000):
        y_pred = (errors > threshold).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        fpr = fp / (fp + tn)  # False Positive Rate
        fnr = fn / (fn + tp)  # False Negative Rate

        diff = abs(fpr - fnr)

        if diff < min_diff:
            min_diff = diff
            optimal_threshold = threshold

    return optimal_threshold

threshold = find_optimal_threshold(reconstruction_errors, y_test)


In [18]:
y_pred = (reconstruction_errors > threshold).astype(int)


In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Autoencoder Classifier Metrics:\nAccuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}")



Autoencoder Classifier Metrics:
Accuracy: 0.4909344490934449
Precision: 0.4553072625698324
Recall: 0.4894894894894895
F1 Score: 0.47178002894356


In [21]:
from sklearn.linear_model import LogisticRegression

# Train the logistic regression model with an increased number of iterations
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

# Make predictions
y_pred_log_reg = log_reg.predict(X_test)

# Calculate metrics
accuracy_log_reg = accuracy_score(y_test, y_pred_log_reg)
precision_log_reg = precision_score(y_test, y_pred_log_reg)
recall_log_reg = recall_score(y_test, y_pred_log_reg)
f1_log_reg = f1_score(y_test, y_pred_log_reg)

print(f"Logistic Regression Metrics:\nAccuracy: {accuracy_log_reg}\nPrecision: {precision_log_reg}\nRecall: {recall_log_reg}\nF1 Score: {f1_log_reg}")


Logistic Regression Metrics:
Accuracy: 0.899116689911669
Precision: 0.895748987854251
Recall: 0.8858858858858859
F1 Score: 0.890790135883241


In [22]:
from sklearn.ensemble import RandomForestClassifier

# Train the random forest model
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# Make predictions
y_pred_rf = rf.predict(X_test)

# Calculate metrics
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)

print(f"Random Forest Metrics:\nAccuracy: {accuracy_rf}\nPrecision: {precision_rf}\nRecall: {recall_rf}\nF1 Score: {f1_rf}")


Random Forest Metrics:
Accuracy: 0.900976290097629
Precision: 0.882295719844358
Recall: 0.9079079079079079
F1 Score: 0.8949185989146522


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Train the random forest model
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# Make predictions
y_pred_rf = rf.predict(X_test)

# Calculate metrics
accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
f1_rf = f1_score(y_test, y_pred_rf)

print(f"Random Forest Metrics:\nAccuracy: {accuracy_rf}\nPrecision: {precision_rf}\nRecall: {recall_rf}\nF1 Score: {f1_rf}")


In [23]:
from sklearn.neural_network import MLPClassifier

#Train
mlp = MLPClassifier(solver='sgd', learning_rate = "adaptive", learning_rate_init = 0.1, alpha=1e-5,hidden_layer_sizes=(64,32), random_state=1,max_iter=500)
mlp.fit(X_train, y_train)

#Predict
y_pred_mlp = mlp.predict(X_test)

# Calculate metrics
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
precision_mlp = precision_score(y_test, y_pred_mlp)
recall_mlp = recall_score(y_test, y_pred_mlp)
f1_mlp = f1_score(y_test, y_pred_mlp)

print(f"MLP:\nAccuracy: {accuracy_mlp}\nPrecision: {precision_mlp}\nRecall: {recall_mlp}\nF1 Score: {f1_mlp}")

MLP:
Accuracy: 0.905625290562529
Precision: 0.8940594059405941
Recall: 0.9039039039039038
F1 Score: 0.8989547038327527


In [24]:
from sklearn.svm import LinearSVC

#Train
svc = LinearSVC(penalty = "l1", loss="squared_hinge",dual=False, max_iter = 10000, tol = 0.001)
svc.fit(X_train, y_train)

#Predict
y_pred_svc = svc.predict(X_test)

# Calculate metrics
accuracy_svc = accuracy_score(y_test, y_pred_svc)
precision_svc = precision_score(y_test, y_pred_svc)
recall_svc = recall_score(y_test, y_pred_svc)
f1_svc = f1_score(y_test, y_pred_svc)

print(f"SVM:\nAccuracy: {accuracy_svc}\nPrecision: {precision_svc}\nRecall: {recall_svc}\nF1 Score: {f1_svc}")

SVM:
Accuracy: 0.898186889818689
Precision: 0.8963414634146342
Recall: 0.8828828828828829
F1 Score: 0.8895612708018154


In [27]:
from sklearn.tree import DecisionTreeClassifier

#Train
dt = DecisionTreeClassifier(random_state=0)
dt.fit(X_train, y_train)

#Predict
y_pred_dt = dt.predict(X_test)

# Calculate metrics
accuracy_dt = accuracy_score(y_test, y_pred_dt)
precision_dt = precision_score(y_test, y_pred_dt)
recall_dt = recall_score(y_test, y_pred_dt)
f1_dt = f1_score(y_test, y_pred_dt)

print(f"Decision Tree:\nAccuracy: {accuracy_dt}\nPrecision: {precision_dt}\nRecall: {recall_dt}\nF1 Score: {f1_dt}")

Decision Tree:
Accuracy: 0.8303114830311483
Precision: 0.8163672654690619
Recall: 0.8188188188188188
F1 Score: 0.8175912043978012


In [28]:
from sklearn.neighbors import KNeighborsClassifier

#Train
knn = KNeighborsClassifier(n_neighbors=5, metric="cosine")
knn.fit(X_train, y_train)

#Predict
y_pred_knn = knn.predict(X_test)

# Calculate metrics
accuracy_knn = accuracy_score(y_test, y_pred_knn)
precision_knn = precision_score(y_test, y_pred_knn)
recall_knn = recall_score(y_test, y_pred_knn)
f1_knn = f1_score(y_test, y_pred_knn)

print(f"KNN:\nAccuracy: {accuracy_knn}\nPrecision: {precision_knn}\nRecall: {recall_knn}\nF1 Score: {f1_knn}")

KNN:
Accuracy: 0.897721989772199
Precision: 0.91129883843717
Recall: 0.8638638638638638
F1 Score: 0.8869475847893115


In [29]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

#Train
rf = RandomForestClassifier(max_depth=10, random_state=0)
ada = AdaBoostClassifier(estimator = rf, n_estimators=10)
ada.fit(X_train, y_train)

#Predict
y_pred_ada = ada.predict(X_test)

# Calculate metrics
accuracy_ada = accuracy_score(y_test, y_pred_ada)
precision_ada = precision_score(y_test, y_pred_ada)
recall_ada = recall_score(y_test, y_pred_ada)
f1_ada = f1_score(y_test, y_pred_ada)

print(f"Decision Tree:\nAccuracy: {accuracy_ada}\nPrecision: {precision_ada}\nRecall: {recall_ada}\nF1 Score: {f1_ada}")

Decision Tree:
Accuracy: 0.9121338912133892
Precision: 0.905811623246493
Recall: 0.9049049049049049
F1 Score: 0.9053580370555834
